이마트 웹사이트는 JavaScript로 렌더링된 콘텐츠이기 때문에, 일반적인 requests 라이브러리로는 데이터가 보이지 않습니다. 따라서 Selenium을 활용하여 데이터를 수집하였습니다. 

https://googlechromelabs.github.io/chrome-for-testing/#stable 에서 다운로드했습니다. 

In [2]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time


ModuleNotFoundError: No module named 'selenium'

In [3]:
pip install -r requirements.txt

Processing d:\home\conda\feedstock_root\build_artifacts\asttokens_1733250440834\work (from -r requirements.txt (line 3))
Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: 'D:\\home\\conda\\feedstock_root\\build_artifacts\\asttokens_1733250440834\\work'



In [4]:
# 크롬 드라이버 경로 설정 (상대경로로 실행하니 오류가 발생합니다. 왜그런지는 잘 모르겠습니다.)
CHROME_DRIVER_PATH = "D:\\github\\angeon-llm\\chromedriver-win64\\chromedriver.exe"

# 크롬 옵션 설정
options = Options()
options.add_argument("--headless")  # 백그라운드 실행
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

try:
    service = Service(CHROME_DRIVER_PATH)
    driver = webdriver.Chrome(service=service, options=options)
    driver.get("https://www.google.com")
    time.sleep(2)
    
    print("✅ 크롬 드라이버가 정상적으로 실행되었습니다.")
    print("페이지 타이틀:", driver.title)

    driver.quit()
except Exception as e:
    print("❌ 크롬 드라이버 실행 실패:")
    print(e)

✅ 크롬 드라이버가 정상적으로 실행되었습니다.
페이지 타이틀: Google


In [20]:
def get_emart_all_tab_sale_items():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(CHROME_DRIVER_PATH), options=options)
    driver.get("https://eapp.emart.com/webapp/product/flyer?trcknCode=main_leaflet")
    time.sleep(10)

    items = []
    visited_titles = set()

    # 1. 모든 탭 버튼 수집
    tab_buttons = driver.find_elements(By.CSS_SELECTOR, ".ern-leaflet-sub-menu button.exhibition-list-item")

    for idx, tab in enumerate(tab_buttons):
        try:
            driver.execute_script("arguments[0].click();", tab)
            time.sleep(3)

            # 2. 해당 탭에서 슬라이드 반복
            while True:
                product_elements = driver.find_elements(By.CSS_SELECTOR, "div.swiper-slide-active div.product-list")

                for product in product_elements:
                    try:
                        title_el = product.find_elements(By.CSS_SELECTOR, "strong.title")
                        if not title_el:
                            continue

                        name = title_el[0].text
                        if name in visited_titles:
                            continue
                        visited_titles.add(name)

                        price_el = product.find_elements(By.CSS_SELECTOR, "div.final-price span.value")
                        price = price_el[0].text if price_el else ""

                        origin_price_el = product.find_elements(By.CSS_SELECTOR, "div.origin-price span.value")
                        original_price = origin_price_el[0].text if origin_price_el else ""

                        discount_el = product.find_elements(By.CSS_SELECTOR, "div.discount")
                        discount = discount_el[0].text if discount_el else ""

                        items.append({
                            "카테고리": tab.text,
                            "상품명": name,
                            "판매가": price,
                            "정상가": original_price,
                            "할인율": discount
                        })

                    except Exception as e:
                        print(f"상품 처리 중 오류: {e}")
                        continue

                # 슬라이드 넘기기
                next_button = driver.find_elements(By.CSS_SELECTOR, "button.swiper-button-next")
                if not next_button or not next_button[0].is_enabled():
                    break  # 다음 슬라이드가 없다면 루프 종료

                driver.execute_script("arguments[0].click();", next_button[0])
                time.sleep(1.5)

        except Exception as e:
            print(f"탭 {idx} ({tab.text}) 처리 중 오류: {e}")
            continue

    driver.quit()
    return pd.DataFrame(items)


In [ ]:
df_all = get_emart_all_tab_sale_items()
df_all.to_csv("emart_flyer_fruits.csv", index=False, encoding="utf-8-sig")
df_all.head()


,카테고리,상품명,판매가,정상가,할인율
0,전체,파프리카 3입 (봉),"2,670","4,980",할인율\n46\n%
1,전체,햇양파 (1.8kg/망),"2,864","3,580",할인율\n20\n%
2,전체,반전참외 (15입내/봉),"8,800","11,000",할인율\n20\n%
3,전체,대추방울토마토 (1kg/팩)/하우스감귤 (800g/팩),"10,980/16,900\n8,784/13,520",,
4,전체,흙당근 (1kg/봉)/햇 세척인삼 (750g/팩),"3,180/37,480\n2,544/29,984",,


상품 링크가 추가된 코드

In [8]:
def get_emart_all_tab_sale_items():
    options = Options()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Chrome(service=Service(CHROME_DRIVER_PATH), options=options)
    driver.get("https://eapp.emart.com/webapp/product/flyer?trcknCode=main_leaflet")
    time.sleep(10)

    items = []
    visited_titles = set()

    tab_buttons = driver.find_elements(By.CSS_SELECTOR, ".ern-leaflet-sub-menu button.exhibition-list-item")

    for idx, tab in enumerate(tab_buttons):
        try:
            driver.execute_script("arguments[0].click();", tab)
            time.sleep(3)

            while True:
                product_elements = driver.find_elements(By.CSS_SELECTOR, "div.swiper-slide-active div.product-list")

                for product in product_elements:
                    try:
                        title_el = product.find_elements(By.CSS_SELECTOR, "strong.title")
                        if not title_el:
                            continue

                        name = title_el[0].text
                        if name in visited_titles:
                            continue
                        visited_titles.add(name)

                        # ✅ 상품 ID 추출: img[data-img] 속성
                        img_el = product.find_elements(By.CSS_SELECTOR, "div.product-img img")
                        product_link = ""
                        if img_el:
                            img_url = img_el[0].get_attribute("data-img") or img_el[0].get_attribute("src")
                            # 상품 ID 추출
                            import re
                            match = re.search(r'/item/(\d+)', img_url)
                            if match:
                                item_id = match.group(1)
                                product_link = f"https://www.ssg.com/item/itemView.ssg?itemId={item_id}"

                        price_el = product.find_elements(By.CSS_SELECTOR, "div.final-price span.value")
                        price = price_el[0].text if price_el else ""

                        origin_price_el = product.find_elements(By.CSS_SELECTOR, "div.origin-price span.value")
                        original_price = origin_price_el[0].text if origin_price_el else ""

                        discount_el = product.find_elements(By.CSS_SELECTOR, "div.discount")
                        discount = discount_el[0].text if discount_el else ""

                        items.append({
                            "카테고리": tab.text,
                            "상품명": name,
                            "상품링크": product_link,
                            "판매가": price,
                            "정상가": original_price,
                            "할인율": discount
                        })

                    except Exception as e:
                        print(f"상품 처리 중 오류: {e}")
                        continue

                next_button = driver.find_elements(By.CSS_SELECTOR, "button.swiper-button-next")
                if not next_button or not next_button[0].is_enabled():
                    break

                driver.execute_script("arguments[0].click();", next_button[0])
                time.sleep(1.5)

        except Exception as e:
            print(f"탭 {idx} ({tab.text}) 처리 중 오류: {e}")
            continue

    driver.quit()
    return pd.DataFrame(items)


In [9]:
df = get_emart_all_tab_sale_items()
df.to_csv("emart_all_sale_items_with_links.csv", index=False, encoding="utf-8-sig")
df.head()


,카테고리,상품명,상품링크,판매가,정상가,할인율
0,전체,파프리카 3입 (봉),https://www.ssg.com/item/itemView.ssg?itemId=1...,"2,670","4,980",할인율\n46\n%
1,전체,햇양파 (1.8kg/망),https://www.ssg.com/item/itemView.ssg?itemId=0...,"2,864","3,580",할인율\n20\n%
2,전체,반전참외 (15입내/봉),https://www.ssg.com/item/itemView.ssg?itemId=0...,"8,800","11,000",할인율\n20\n%
3,전체,대추방울토마토 (1kg/팩)/하우스감귤 (800g/팩),,"10,980/16,900\n8,784/13,520",,
4,전체,흙당근 (1kg/봉)/햇 세척인삼 (750g/팩),,"3,180/37,480\n2,544/29,984",,
